In [ ]:
!pip install langgraph

In [7]:
greetings_pool = [["salve", "hello"],
 ["ciao", "hi"],
 ["bounaserra", "good evening"],
 ["buongiorno", "good morning"]]

## State

We will define a graph.

Our state will be a dict.

We can access this from any graph node as state['keys'].

In [8]:
from typing import List, Dict, TypedDict
from pydantic import BaseModel, Field

In [9]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

## Nodes

Each node will simply modify the state.

In [35]:
def hello(state):
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    print("System Greetings")
    print(greetings)
    choice = random.randint(0,3)
    greetings = greetings_pool[choice]
    print("----HELLO-----")
    print(greetings)
    return {
        "keys": {"greetings": greetings}
    }

In [36]:
import random
def hello_back(state):
    print("------HELLO BACK---------")
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    choice = random.randint(0,3)
    return_greetings = greetings_pool[choice]
    print(return_greetings)
    return {
        "keys": {"greetings": greetings, "return_greetings":return_greetings}
    }

## Edges

Each edge will choose which node to call next.

In [37]:
def greetings_grader(state):
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    return_greetings = state_dict["return_greetings"]
    grade = "good greeter"
    for greeting in return_greetings:
        if greeting in greetings:
            continue
        grade = "bad greeter"
        break
    print("-----Greetings Grader-----")
    if grade == "good greeter":
        print("DECISION-----GOOD GREETER----GOODBYE")
    else:
        print("DECISION-----BAD GREETER----GREET AGAIN")
    return grade

## Build and Compile Graph

In [38]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(GraphState)

# Define the two nodes we will cycle between
workflow.add_node("hello", hello)
workflow.add_node("hello_back", hello_back)

# Set the entrypoint as `hello`
workflow.set_entry_point("hello")

# We now add a conditional edge
# We grade the greetings from hello-back
# If grade is "good greeter", we end or we cycle
# back to hello-back
workflow.add_conditional_edges(

    "hello_back",
    greetings_grader,
    {
        "bad greeter": "hello_back",
        "good greeter": END
    }
)

workflow.add_edge('hello', 'hello_back')

app = workflow.compile()

type(app)

langgraph.pregel.Pregel

## Run Graph

In [45]:
greetings = ["Hello you two, greet politely"]
try:
    app.invoke({"keys":{"greetings": greetings}}, config={"recursion_limit": 10})
except:
    print("Oh no, you reached the greeting limit!")

System Greetings
['Hello you two, greet politely']
----HELLO-----
['buongiorno', 'good morning']
------HELLO BACK---------
['ciao', 'hi']
-----Greetings Grader-----
DECISION-----BAD GREETER----GREET AGAIN
------HELLO BACK---------
['bounaserra', 'good evening']
-----Greetings Grader-----
DECISION-----BAD GREETER----GREET AGAIN
------HELLO BACK---------
['buongiorno', 'good morning']
-----Greetings Grader-----
DECISION-----GOOD GREETER----GOODBYE
